In [1]:
from math import sqrt
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_ieee = pd.read_csv("SOURCE/IEEE_Xplore.csv", sep=',', quotechar='"')
df_ieee['Publication Title'] = df_ieee['Publication Title'].str.replace(r'^[0-9]* ', '')
df_ieee['Publication Title'] = df_ieee['Publication Title'].str.replace(r'[0-9]+(?:st| st|nd| nd|rd| rd|th| th) ', '')
pd.DataFrame(df_ieee.groupby('Publication Title')['Document Title'].count().sort_values(ascending=False)).to_csv("COUNT/IEEE_Xplore_count.csv")

<ipython-input-3-02d2bf4ca36d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ieee['Publication Title'] = df_ieee['Publication Title'].str.replace(r'^[0-9]* ', '')
<ipython-input-3-02d2bf4ca36d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ieee['Publication Title'] = df_ieee['Publication Title'].str.replace(r'[0-9]+(?:st| st|nd| nd|rd| rd|th| th) ', '')


In [2]:
df_acm = pd.read_csv("SOURCE/ACM_cleaned.csv", header=None)
df_acm[0] = df_acm[0].str.replace("Proceedings of the [0-9]+(?:st| st|nd| nd|rd| rd|th| th) ","")
df_acm[0] = df_acm[0].str.replace("Proceedings of the [0-9][0-9][0-9][0-9] ","")
df_acm[0] = df_acm[0].str.replace(" \d+$","")
pd.DataFrame(df_acm.groupby(0)[0].count().sort_values(ascending=False)).to_csv("COUNT/ACM_count.csv")

<ipython-input-2-d5d20eab38a3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_acm[0] = df_acm[0].str.replace("Proceedings of the [0-9]+(?:st| st|nd| nd|rd| rd|th| th) ","")
<ipython-input-2-d5d20eab38a3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_acm[0] = df_acm[0].str.replace("Proceedings of the [0-9][0-9][0-9][0-9] ","")
<ipython-input-2-d5d20eab38a3>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_acm[0] = df_acm[0].str.replace(" \d+$","")


In [5]:
df_springer = pd.read_csv("SOURCE/Springer.csv", sep=',')
pd.DataFrame(df_springer.groupby("Publication Title")["Publication Title"].count().sort_values(ascending=False)).to_csv("COUNT/Springer_count.csv")

In [7]:
df_sciencedirect = pd.read_csv("SOURCE/ScienceDirect.csv", sep=',', header=None)
pd.DataFrame(df_sciencedirect.groupby(0)[0].count().sort_values(ascending=False)).to_csv("COUNT/ScienceDirect_count.csv")

In [11]:
df_acl = pd.read_csv("ACL_Anthology.csv", sep=',', escapechar='\\')
df_acl['Year']=pd.to_numeric(df_acl["Year"])
df_acl = df_acl[df_acl['Year'] >= 2017]
df_acl

cols_to_filter = ['Abstract', 'Title']
search_values = ["topic label", "topic labels", "topic labeling", "topic labelling"]

patt = '|'.join(search_values)

mask = df_acl[cols_to_filter].apply(lambda x: x.str.contains(patt, case=False)).any(1)
df_acl = df_acl[mask].copy()

df_acl['Venue'] = df_acl['Venue'].str.replace("Proceedings of the [0-9]+(?:st| st|nd| nd|rd| rd|th| th) ","")
df_acl['Venue'] = df_acl['Venue'].str.replace("Proceedings of the [0-9][0-9][0-9][0-9] ","")
df_acl.groupby('Venue')['Venue'].count().sort_values(ascending=False).to_csv("ACL_Anthology_count.csv")

<ipython-input-11-2392329de022>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df_acl['Venue'] = df_acl['Venue'].str.replace("Proceedings of the [0-9]+(?:st| st|nd| nd|rd| rd|th| th) ","")
<ipython-input-11-2392329de022>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df_acl['Venue'] = df_acl['Venue'].str.replace("Proceedings of the [0-9][0-9][0-9][0-9] ","")


Venue
Proceedings of the International Conference on Recent Advances in Natural Language Processing (RANLP 2021)       2
Conference of the European Chapter of the Association for Computational Linguistics: Main Volume                 1
Conference of the {E}uropean Chapter of the Association for Computational Linguistics: Volume 2, Short Papers    1
Findings of the Association for Computational Linguistics: EMNLP 2021                                            1
International Conference on Natural Language Generation                                                          1
Proceedings of the The 19th Annual Workshop of the Australasian Language Technology Association                  1
Name: Venue, dtype: int64

In [10]:
pd.DataFrame(df_acl.groupby('Venue')['Venue'].count().sort_values(ascending=False))

,Venue
Venue,
Proceedings of the International Conference on Recent Advances in Natural Language Processing (RANLP 2021),2
Conference of the European Chapter of the Association for Computational Linguistics: Main Volume,1
"Conference of the {E}uropean Chapter of the Association for Computational Linguistics: Volume 2, Short Papers",1
Findings of the Association for Computational Linguistics: EMNLP 2021,1
International Conference on Natural Language Generation,1
Proceedings of the The 19th Annual Workshop of the Australasian Language Technology Association,1
